Simulation of N-S equation using PINNS:
a) nn to model velocity(u,v,w) and pressure p
b) loss function enforcing continuity and momentum eqns
c) training loop with Adam optimizer

Necessary imports

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

Define the neural network

In [3]:
class PINN(nn.Module):
  def _init_(self):
    super(PINN,self)._init_()
    self.layers = nn.Sequential(
        nn.Linear(3,128),
        nn.Tanh(),
        nn.Linear(128,128),
        nn.Tanh(),
        nn.Linear(128,128),
        nn.Tanh(),
        nn.Linear(128, 4)     #outputs u, v, w, p
    )

    def forward(self, x):
      return self.layers(x)

Physics informed Loss Function

In [4]:
def pinn_loss(model, collocation_points, boundary_points, boundary_values, Re):
  u,v,w,p=torch.split(model(collocation_points),1, dim=1)

  #derivatives
  grads = lambda f,x: torch.autograd.grad(f,x,grad_outputs=torch.ones_like(f),create_graph=True)[0]

  du_dx=grads(u,collocation_points)[:, 0:1]
  du_dy=grads(u, collocation_points)[:,1:2]
  du_dz=grads(u, collocation_points)[:,2:3]

  dv_dx = grads(v, collocation_points)[:, 0:1]
  dv_dy = grads(v, collocation_points)[:, 1:2]
  dv_dz = grads(v, collocation_points)[:, 2:3]

  dw_dx = grads(w, collocation_points)[:, 0:1]
  dw_dy = grads(w, collocation_points)[:, 1:2]
  dw_dz = grads(w, collocation_points)[:, 2:3]

  dp_dx = grads(p, collocation_points)[:, 0:1]
  dp_dy = grads(p, collocation_points)[:, 1:2]
  dp_dz = grads(p, collocation_points)[:, 2:3]

  #continuity eqn
  continuity = du_dx + dv_dy +dw_dz

  #momentum eqn
  momentum_x = u * du_dx + v * du_dy + w * du_dz + dp_dx - (1 / Re) * (grads(du_dx, collocation_points)[:, 0:1] + grads(du_dy, collocation_points)[:, 1:2] + grads(du_dz, collocation_points)[:, 2:3])
  momentum_y = u * dv_dx + v * dv_dy + w * dv_dz + dp_dy - (1 / Re) * (grads(dv_dx, collocation_points)[:, 0:1] + grads(dv_dy, collocation_points)[:, 1:2] + grads(dv_dz, collocation_points)[:, 2:3])
  momentum_z = u * dw_dx + v * dw_dy + w * dw_dz + dp_dz - (1 / Re) * (grads(dw_dx, collocation_points)[:, 0:1] + grads(dw_dy, collocation_points)[:, 1:2] + grads(dw_dz, collocation_points)[:, 2:3])

  #boundary loss
  u_boundary, v_boundary, w_boundary, p_boundary = torch.split(model(boundary_points), 1, dim=1)
  boundary_loss = torch.mean((u_boundary - boundary_values[:, 0:1])**2 + (v_boundary - boundary_values[:, 1:2])**2 +
                               (w_boundary - boundary_values[:, 2:3])**2 + (p_boundary - boundary_values[:, 3:4])**2)

  # Combine Loss
  loss = torch.mean(continuity**2) + torch.mean(momentum_x**2) + torch.mean(momentum_y**2) + torch.mean(momentum_z**2) + boundary_loss
  return loss

Training Process

In [6]:
if __name__=="_main_":
  #collocation points (x, y,z)
  collocation_points = torch.rand((10000,3), redquires_grad=True)

  #boundary points and values
  boundary_points=torch((1000,3), requires_grad=True)
  boundary_values=torch.zeros((1000,4)) #modify based on boundary conditions

  #Reynodls number
  Re=100

  #model initialization
  model=PINN()
  optimizer=optim.Adam(model.parameters(),lr=0.001)

  #training loop
  for epoch in range(10000):
    optimizer.zero_grad()
    loss=pinn.loss(model, collocation_points, boundary_points, boundary_values, Re)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
      print(f"Epoch{epoch}, loss: {loss.item()}")

  #save model
  torch.save(model.state_dict(), "pinn_model.pt")
  print("Model Trining complete")